In [ ]:
import sqlite3
import numpy as np
import pandas as pd
import ctypes
import shutil
import os
MessageBox = ctypes.windll.user32.MessageBoxW
from System_Assessment_Generate_Models_Variables import *


In [ ]:
#FSA System Assessment
model_area = 'FSA'
model_original = r"J:\SEWER_AREA_MODELS\FSA\01_MASTER_MODEL\MODEL\FSA_Base_2021pop.sqlite"
year_original = 2021
version = 103

year_scenario_list = []
year_scenario_list.append([2034,'2030_Network',['Ex']])
# year_scenario_list.append([2040,'2030_Network',['2050M','2050H']])
# year_scenario_list.append([2051,'2030_Network',['2050H']])
# year_scenario_list.append([2060,'2030_Network',['2050H','2100H']])

population_sheet = r"J:\SEWER_AREA_MODELS\FSA\02_MODEL_COMPONENTS\04_DATA\01. POPULATION\FSA_Master_Population_File_Update14a15a.xlsx"
population_tab = 'MPF Update 14a'
all_scenarios = True
dual_catchments = False
use_all_files_in_folder = False
output_folder = r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\System_Assessment\Model_Generation'
script_path = r'J:\TOOLS\Generate_WaterspillDischarge_C#\WaterspillDischarge.cs'

In [ ]:
def sql_to_df(sql,model):
  con = sqlite3.connect(model)
  df = pd.read_sql(sql, con)
  con.close()
  return df

def execute_sql(sql,model):
  con = sqlite3.connect(model)
  cur = con.cursor()
  cur.execute(sql)
  cur.close()
  con.commit()
  con.close()
    
def df_to_sql(df,table_name,model):
    conn = sqlite3.connect(model)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()
    

In [ ]:
keep_cols = ['Catchment','Zone','Year','Pop_ResLD','Pop_ResHD','Pop_Mixed','Area_Com','Area_Ind','Area_Inst']
pop_df = pd.read_excel(population_sheet,sheet_name=population_tab,usecols=keep_cols,dtype={'Catchment': str})

# pop_df = pop_df[pop_df.Catchment!='84394S']
# pop_df = pop_df[pop_df.Catchment!='84279']
# pop_df = pop_df.drop(pop_df.index[-1])

# Melt the DataFrame to combine the columns
columns_to_combine = ['Pop_ResLD', 'Pop_ResHD', 'Pop_Mixed', 'Area_Com', 'Area_Ind', 'Area_Inst']
pop_df_melt = pd.melt(pop_df, id_vars=['Catchment', 'Zone', 'Year'], value_vars=columns_to_combine, var_name='Type', value_name='Value')
pop_df_melt[['Major_type', 'Minor_type']] = pop_df_melt['Type'].str.split('_', n=1, expand=True)
pop_df_melt = pop_df_melt.drop(columns=['Type'])

In [ ]:
dir(pop_df_melt['Type'].str)

In [ ]:
#Check for missing catchments
catchment_years = []
sql = "SELECT catchmentid FROM msm_Loadpoint GROUP BY catchmentid"
muids = list(sql_to_df(sql,model_original).catchmentid)

for muid in muids:
    for year_scenario in year_scenario_list:
        year = year_scenario[0]
        catchment_years.append([muid,year])
catchment_year_df = pd.DataFrame(catchment_years,columns=(['Catchment','Year']))
catchment_year_df 

merged = catchment_year_df.merge(pop_df[['Catchment', 'Year']], on=['Catchment', 'Year'], how='left', indicator=True)

not_founds = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])

if len(not_founds) > 0:
    message = "WARNING.The following catchment/year combinations are not found\n\n"
    for index, row in not_founds.iterrows():
        message += row[0] + ', ' + str(row[1]) + '.\n'
    message += '\nContinue?'
    
    if MessageBox(None, message, 'Warning', 4) == 7:
        MessageBox(None, "Please report the missing catchment(s)", 'Info', 0)
        raise ValueError(message)
    else:
        pass

for year_scenario in year_scenario_list:
    year = year_scenario[0]
    scenario = year_scenario[1]
    turnons = year_scenario[2]
      
    model_name = model_area + '_' + str(year) + 'pop_V' + str(version) + '.sqlite'

    if os.path.basename(model_original) == model_name:

        message = "Tool ends. For year " + str(year) + ", the new model name '" + model_name + "' is the same as the original."
        MessageBox(None, message, 'Info', 0)
        raise ValueError("message")

    #Delete sqlite if it exists and create a new
    mu_path = output_folder + "\\" + model_name
    os.remove(mu_path) if os.path.exists(mu_path) else None
    shutil.copyfile(model_original, mu_path)
    
    script_path_new = os.path.splitext(mu_path)[0] + '.cs'
    shutil.copy(script_path, script_path_new)
    
    mupp_path_original = os.path.splitext(model_original)[0] + '.mupp'
    mupp_path_new = os.path.splitext(mu_path)[0] + '.mupp'
    
    with open(mupp_path_original, 'r') as source_file, open(mupp_path_new, 'w') as output_file:
        for line in source_file:
            if line.startswith(r'   DBFilePath = |.'):
                output_file.write('   DBFilePath = |.\\' + os.path.splitext(model_name)[0] + '.sqlite|\n')
            else:
                output_file.write(line)
    

    sql = "SELECT MUID FROM msm_Project WHERE enable_hd = 1"
    muids = list(sql_to_df(sql,mu_path).muid)
    for muid in muids:
        muid_new = muid.replace(str(year_original) + 'p',str(year) + 'p')
        sql = "UPDATE msm_Project SET scenarioname = '" + scenario + "', muid = '" + muid_new + "' WHERE muid = '" + muid + "'"
        execute_sql(sql, mu_path)
        
        sql = "UPDATE msm_ProjectOutput SET simulationid = '" + muid_new + "' WHERE simulationid = '" + muid + "'"
        execute_sql(sql, mu_path)
        

    sql = "UPDATE msm_Project SET IncludeToBatchNo = 0 WHERE MUID LIKE '%h-AES_%'"
    execute_sql(sql, mu_path)

    for turnon in turnons:
        sql = "UPDATE msm_Project SET IncludeToBatchNo = 1 WHERE MUID LIKE '%h-AES_%' AND MUID LIKE '%" + turnon + "%'"
        execute_sql(sql, mu_path)
        
        
    pop_df_melt_year = pop_df_melt[pop_df_melt.Year==year]
    df_to_sql(pop_df_melt_year,'New_Population',mu_path)
    
       
# del mu_path   
    

In [ ]:
df_to_sql(pop_df_melt_year,'New_Population',mu_path)

In [ ]:
import sqlite3
import pandas as pd

mu_path = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\System_Assessment\Model_Generation\FSA_2033pop_V103 - Copy.sqlite"

# Sample DataFrame
data = {'ID': [1, 2, 3],
        'Name': ['Alice', 'Bob', 'Charlie']}
df = pd.DataFrame(data)

# Define the SQLite database filename


# Establish a connection to the SQLite database
conn = sqlite3.connect(mu_path)

# Export the DataFrame to an SQLite table
table_name = 'MyTables'  # Name of the table in the database
# pop_df_melt_year[['Zone','Value']].to_sql(table_name, conn, if_exists='replace', index=False)
df.to_sql(table_name, conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

print(f"DataFrame exported to '{table_name}' in '{mu_path}'")


In [ ]:
del conn

In [ ]:
# Sample DataFrame
data = {'ID': [1, 2, 3],
        'Name': ['Alice', 'Bob', 'Charlie']}
df = pd.DataFrame(data)

# Define the SQLite database filename
db_filename = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\System_Assessment\Model_Generation\FSA_2033pop_V103 - Copy.sqlite"


try:
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_filename)

    # Export the DataFrame to an SQLite table
    table_name = 'my_tabless'  # Name of the table in the database
    df.to_sql(table_name, conn, if_exists='replace', index=False)

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    print(f"DataFrame exported to '{table_name}' in '{db_filename}'")
except Exception as e:
    print(f"Error: {str(e)}")